In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# import
import pandas as pd
from pprint import pprint
from zen_garden.postprocess.results import Results
from pathlib import Path
from os import chdir

# set path
path = Path('/Users/davidbertschinger/Documents/GitHub/ZEN-garden/notebooks')
chdir(path)

In [13]:
# define import path
path_import = Path("../data/outputs/05_CCTS_WTE_small_01_cost_optimal")

# define export path
path_export = Path("/Users/davidbertschinger/Documents/GitHub/ZEN-garden/data/05_CCTS_WTE_small_02_n1")

In [14]:
# get cost optimal solution
results = Results(path_import)

In [15]:
# get capacity addition and save as csv
results.get_df('capacity_addition', to_csv="capacity_addition")

technology     capacity_type  location                     year
capture_amine  power          swiss_storage_CH             0        0.000000
                              waste_11_CH                  0       22.360440
                              waste_19_CH                  0       30.584139
                              waste_1_CH                   0       26.387584
                              waste_2_CH                   0       19.367213
                                                                     ...    
truck_batch    power          waste_9_CH-swiss_storage_CH  0       10.046834
                              waste_9_CH-waste_11_CH       0        0.000000
                              waste_9_CH-waste_19_CH       0        0.000000
                              waste_9_CH-waste_1_CH        0        0.000000
                              waste_9_CH-waste_2_CH        0        0.000000
Name: capacity_addition, Length: 129, dtype: float64

In [16]:
# get capacity addition csv
df = pd.read_csv('capacity_addition.csv')

In [17]:
# create existing capacity files

# find unique technologies
technology = df['technology'].unique()

for i in range(len(technology)):
     
    #Get table of one technology 
    tec = technology[i]
    x = df[df['technology'] == tec]
    
    #Set path
    parent_path = Path(path_export)
    path = [file for file in Path(parent_path).rglob(tec)]
    path = path[0]
    filepath = Path(str(path)+'/capacity_existing.csv')
    
    #Clean up table
    x = x.drop(columns='capacity_type')
    x = x.drop(columns='technology')
    x = x[x['capacity_addition'] != 0]
    x = x.rename(columns={'year': 'year_construction'})
    x = x.rename(columns={'capacity_addition': 'capacity_existing'})
    x['year_construction'] = x['year_construction'] + 2025 
    
    #clean up table, differentiate between node and edge
    if path.parent.name == 'set_transport_technologies':
        x = x.rename(columns={'location': 'edge'})

    else:
        x = x.rename(columns={'location': 'node'})

    #Safe
    x.to_csv(filepath, index=False)